In [1]:
import os
import cv2
import time
import numpy as np
import face_recognition
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn import svm

# Training the SVC classifier

In [2]:
aug = True
if aug:
    data_path = './datasets/temp/'
    file_encoded = 'face_encoded_aug.npy'
    predict_save_name = 'predicted_compare_aug.jpg'
else:
    data_path = './datasets/single/'
    file_encoded = 'face_encoded.npy'
    predict_save_name = 'predicted_compare.jpg'


# The training data would be all the face encodings from all the known images and the labels are their names

# Training directory
train_dir = os.listdir(data_path)
threshold = 0.95

In [3]:
def importFaceEncoded(filename='face_encoded.npy'):
    # with open(filename, 'rb') as f:
    encodings = np.load(filename)
    print(filename + ' has imported!')
    return encodings

In [4]:
def findEncodings():
    # Loop through each person in the training directory
    encodings, names = [], []
    for person in train_dir:
        if person != '.DS_Store':
            pix = os.listdir(data_path + person)
            # Loop through each training image for the current person
            for person_img in pix:
                # Get the face encodings for the face in each image file
                face = face_recognition.load_image_file(
                    data_path + person + "/" + person_img)

                face_enc = face_recognition.face_encodings(face)[0]
                # Add face encoding for current image with corresponding label (name) to the training data
                encodings.append(face_enc)
                names.append(person)
    return encodings, names

replace = True
if not os.path.isfile(file_encoded) or replace:
    encodings, names = findEncodings()
    with open(file_encoded, 'wb') as f:
        np.save(f, encodings)
    with open('names.npy', 'wb') as f:
        np.save(f, names)
else:
    encodings = importFaceEncoded(file_encoded)
    names = importFaceEncoded('names.npy')

In [5]:
def draw_box(img, names_predicted, face_locations):
    fontFace = cv2.FONT_HERSHEY_COMPLEX
    count, fontScale, thickness = 0, 0.5, 1
    red, green = (255, 0, 0), (0, 255, 0)
    for name, faceLoc in zip(names_predicted, face_locations):
        tolerance = 0.6
        # faceDis = face_distance(encodeListKnown, encodeFace)
        # matchIndex = np.argmin(faceDis)
        # confidence = 1 - faceDis[matchIndex]
        # if faceDis[matchIndex] <= tolerance:
        color = green
        # if confidence >= threshold:
        #     name = '%s %.4f' % (
        #         classNames[matchIndex].upper()[:8], confidence)
        #     count += 1
        # else:
        #     name = 'Unknown~'
        #     color = red
        y1, x2, y2, x1 = faceLoc
        labelSize = cv2.getTextSize(
            name, fontFace, fontScale, 2)[0]
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        cv2.rectangle(
            img, (x1, y1), (x1+labelSize[0], y1-labelSize[1]-20), color, cv2.FILLED)
        cv2.putText(img, name, (x1, y1-10),
                    fontFace, fontScale, (0, 0, 0), thickness)
    cnt_x1, cnt_y1 = 12, 12
    count_students = '%d student(s)' % len(names_predicted)
    countSize = cv2.getTextSize(
        count_students, fontFace, fontScale, thickness)[0]
    cv2.rectangle(img, (cnt_x1, cnt_y1), (cnt_y1+countSize[0]+18, cnt_y1+countSize[1]+18),
                  green, -1)
    cv2.putText(img, count_students, (cnt_x1+8, countSize[1]+cnt_y1+8), fontFace,
                fontScale=fontScale, color=(0, 0, 0), thickness=thickness)
    return img

In [7]:
# Create and train the SVC classifier
clf = svm.SVC(C=1000, gamma='scale')
clf.fit(encodings, names)

# Load the test image with unknown faces into a numpy array
test_image = face_recognition.load_image_file('datasets/validation/class_hai.jpg')

# Find all the faces in the test image using the default HOG-based model
face_locations = face_recognition.face_locations(test_image)
no = len(face_locations)
print("Number of faces detected: ", no)


# Predict all the faces in the test image using the trained classifier
print("Found:")
names_predicted = []
for i in range(no):
    test_image_enc = face_recognition.face_encodings(test_image)[i]
    name = clf.predict([test_image_enc])
    names_predicted.append(*name)
    # print(*name)

test_image = draw_box(test_image, names_predicted, face_locations)
plt.imsave(predict_save_name, test_image)
print('\nDONE.')

Number of faces detected:  23
Found:

DONE.
